# 5. LLM API 기초 실습

## 학습 목표
- curl과 Python requests로 LLM API 직접 호출하기
- API 파라미터(temperature, max_tokens 등) 이해하기
- 스트리밍 응답 처리하기

## 1. 환경 설정

In [ ]:
!pip install requests python-dotenv -q

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ .env 파일에 OPENAI_API_KEY를 설정하세요")
else:
    print("✅ API 키 로드 완료")

## 2. 기본 API 호출

### curl 명령어 (터미널에서 실행)
```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [{"role": "user", "content": "Hello!"}]
  }'
```

### Python requests로 호출

In [ ]:
def call_openai(prompt: str) -> str:
    url = "https://api.openai.com/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}]
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]

# 테스트
result = call_openai("Python에서 리스트와 튜플의 차이는?")
print(result)

## 3. API 파라미터 실험

### 주요 파라미터
- **model**: 사용할 모델 (gpt-3.5-turbo, gpt-4 등)
- **temperature**: 창의성 조절 (0.0=결정론적, 2.0=매우 창의적)
- **max_tokens**: 최대 생성 토큰 수
- **messages**: 대화 히스토리 (system, user, assistant)

In [ ]:
def call_openai_advanced(prompt, temperature=0.7, max_tokens=150):
    url = "https://api.openai.com/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    response = requests.post(url, headers=headers, json=data)
    result = response.json()
    
    return {
        "content": result["choices"][0]["message"]["content"],
        "usage": result["usage"]
    }

# Temperature 비교
prompt = "AI에 대한 한 문장 설명을 써줘"

print("Temperature = 0.2 (일관적)")
print(call_openai_advanced(prompt, temperature=0.2)["content"])

print("\nTemperature = 1.5 (창의적)")
print(call_openai_advanced(prompt, temperature=1.5)["content"])

## 4. 시스템 프롬프트와 멀티턴 대화

In [ ]:
def chat_multiturn(messages):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    data = {"model": "gpt-3.5-turbo", "messages": messages}
    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]

# 대화 시작
conversation = [
    {"role": "system", "content": "너는 친절한 Python 선생님이야."},
    {"role": "user", "content": "내 이름은 철수야"}
]

response1 = chat_multiturn(conversation)
print(f"Bot: {response1}\n")
conversation.append({"role": "assistant", "content": response1})

# 이전 대화 기억
conversation.append({"role": "user", "content": "내 이름이 뭐였지?"})
response2 = chat_multiturn(conversation)
print(f"Bot: {response2}")

## 5. 스트리밍 응답

긴 응답을 실시간으로 받을 때 유용합니다.

In [ ]:
def call_openai_streaming(prompt):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "stream": True
    }
    
    response = requests.post(url, headers=headers, json=data, stream=True)
    
    for line in response.iter_lines():
        if line:
            line = line.decode('utf-8')
            if line.startswith("data: ") and line.strip() != "data: [DONE]":
                try:
                    data = json.loads(line[6:])
                    delta = data["choices"][0].get("delta", {})
                    if "content" in delta:
                        print(delta["content"], end="", flush=True)
                except:
                    pass
    print()  # 줄바꿈

# 테스트
call_openai_streaming("Python의 장점 3가지를 설명해줘")

## 6. 실습 과제

### Level 1
1. temperature를 0.0, 0.5, 1.0, 1.5로 바꿔가며 "로봇에 대한 이야기"를 3번씩 생성해보고 차이점 관찰
2. max_tokens를 20, 50, 100으로 설정하고 응답 길이 비교
3. 시스템 프롬프트를 다양하게 바꿔보기 (해적, 셰익스피어, 과학자 등)

In [ ]:
# 과제 1: Temperature 실험
# TODO: 코드 작성


### Level 2
4. 토큰 사용량 기반 비용 계산 함수 만들기 (GPT-3.5: input $0.0015/1K, output $0.002/1K)
5. 에러 처리 및 재시도 로직 구현하기

In [ ]:
# 과제 2: 비용 계산
def calculate_cost(prompt_tokens, completion_tokens):
    # TODO: 구현
    pass


## 다음 단계

이제 LangChain으로 이런 작업들을 더 쉽게 해봅시다!

👉 [06_langchain_practice.ipynb](06_langchain_practice.ipynb)